<h1 style="text-align:center;">Reporte de futuros y tasas</h1>

In [1]:

import pandas as pd
from datetime import date, timedelta
from futures_rfx import *
from bcra import *
from IPython.display import display, HTML
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

end = date.today()
start = end - timedelta(days=30)

df = get_futures(start,end)
TAMAR_SERIE = get_tamar_serie(start, end ,id_variable=44)
bills = get_bills_rates()
rates = get_implied_rates(df)

volume = df.pivot(index='dateTime',columns='symbol',values=['volume'])
volume.columns = volume.columns.droplevel(0)  # Drop the top level of the MultiIndex

openInterest = df.pivot(index='dateTime',columns='symbol',values=['openInterest'])
openInterest.columns = openInterest.columns.droplevel(0)  # Drop the top level of the MultiIndex
last_run = date.today().strftime('%Y-%m-%d')
print('Ultima actualización de datos:', last_run)

Ultima actualización de datos: 2025-10-30


## ***- Precios y tasas implicitas de Futuros***

In [2]:
# Plot limits
last_futures_close = get_last_futures_close_fx_limits(df)
plot_limits_plotly(last_futures_close)
# Plot rates
plot_smooth_plotly(rates[rates < 100])
# Table display
def table_display(rates):
    rates_display = rates.copy()
    rates_display = rates_display/100
    rates_display.index = rates_display.index.strftime("%Y-%m-%d")
    rates_display = rates_display.style.background_gradient(cmap="YlOrRd")
    rates_display = rates_display.format("{:.1%}")
    display(HTML(rates_display.to_html().replace('<table',
                                '<table style="font-size:16px"')))

def table_display_plotly(rates):
    rates_display = rates.copy()
    rates_display = rates_display / 100
    rates_display.index = rates_display.index.strftime("%Y-%m-%d")

    # Color gradient for the background
    # Plotly has a built-in color scale 'YlOrRd' which is similar to the matplotlib one used in your code.
    color_scale = "YlOrRd"

    # Prepare the data for Plotly
    header = rates_display.columns.tolist()
    cells = [rates_display.index.tolist()]
    for col in header:
        cells.append(rates_display[col].apply(lambda x: f"{x:.1%}").tolist())

    # Create the table with Plotly
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=header,
            fill_color='lightgray',
            font=dict(size=14, color='black'),
            align='center'
        ),
        cells=dict(
            values=cells,
            fill_color=[
                [color_scale] * len(rates_display.index)  # Apply color scale
            ],
            font=dict(size=10, color='black'),
            align='center',
            height=30
        )
    )])

    # Show the Plotly table
    fig.show()


table_display(rates)


symbol,DLR012026,DLR022026,DLR052026,DLR102025,DLR112025,DLR122025
dateTime,,,,,,
2025-09-30,62.7%,56.6%,46.4%,109.8%,82.4%,70.3%
2025-10-01,46.6%,nan%,37.0%,62.1%,57.1%,50.6%
2025-10-02,41.3%,38.7%,33.8%,30.7%,43.9%,43.5%
2025-10-03,38.8%,36.4%,33.4%,25.8%,41.1%,40.2%
2025-10-06,nan%,41.4%,35.9%,39.1%,51.9%,48.1%
2025-10-07,48.0%,nan%,37.4%,42.3%,57.3%,51.9%
2025-10-08,47.1%,43.2%,37.3%,44.1%,59.2%,50.7%
2025-10-09,37.9%,35.4%,32.9%,-7.1%,36.6%,38.7%
2025-10-13,46.8%,43.0%,36.5%,21.6%,49.1%,48.0%


## ***- Tasas de referencia: TAMAR y LECAPs a tasa fija***

In [3]:
plot_tamar_serie_plotly(TAMAR_SERIE/100)
plot_tamar_serie_plotly(bills[['expiration_date', 'tna']].set_index('expiration_date'), 'TNA Letras del Tesoro a Tasa Fija', 'Fecha de vencimiento', 'TNA %')

## ***- Interes Abierto y Volumen diario***


In [4]:

plot_futures_plotly(data=volume, kind='bar', ylabel="Volumen", title="Evolución del volumen (30 dias)", legend_name = "Volumen")
plot_futures_plotly(data=openInterest, kind='bar', ylabel="Interes Abierto", title="Evolucion de Interes Abierto (30 dias)", legend_name = "Interes Abierto")

In [5]:
#!py -3.10 -m nbconvert --no-input --to html index.ipynb -output index.html